<a href="https://colab.research.google.com/github/elichen/aoc2024/blob/main/Day_16_Reindeer_Maze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Example usage
input = """#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################"""

In [18]:
input = open("input.txt").read()

In [19]:
from collections import defaultdict
from typing import Dict, List, Set, Tuple

def solve_maze(maze_str: str):
    # Parse the maze into a 2D grid
    grid = [list(line) for line in maze_str.splitlines()]
    height, width = len(grid), len(grid[0])

    # Find start and end positions
    start = end = None
    for i in range(height):
        for j in range(width):
            if grid[i][j] == 'S':
                start = (i, j)
            elif grid[i][j] == 'E':
                end = (i, j)

    def is_intersection(i: int, j: int) -> bool:
        """Check if position is an intersection, dead end, or turn"""
        if grid[i][j] == '#':
            return False

        # Get available directions
        available = []
        for di, dj in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            ni, nj = i + di, j + dj
            if 0 <= ni < height and 0 <= nj < width and grid[ni][nj] != '#':
                available.append((di, dj))

        if len(available) != 2:
            return True  # Intersection or dead end

        # Check if it's a turn by seeing if the two available directions are perpendicular
        (d1i, d1j), (d2i, d2j) = available
        return d1i * d2i + d1j * d2j == 0  # Dot product is 0 for perpendicular vectors

    def find_next_node(i: int, j: int, prev_i: int, prev_j: int, direction: str) -> Tuple[Tuple[int, int], int, str]:
        """Follow path to next node, return node position, distance, and final direction"""
        dist = 0
        curr_i, curr_j = i, j
        curr_dir = direction

        while True:
            if (curr_i, curr_j) != (i, j) and (
                is_intersection(curr_i, curr_j) or
                grid[curr_i][curr_j] in 'SE'
            ):
                return (curr_i, curr_j), dist, curr_dir

            found_next = False
            for di, dj, new_dir in [(0, 1, 'E'), (1, 0, 'S'), (0, -1, 'W'), (-1, 0, 'N')]:
                ni, nj = curr_i + di, curr_j + dj
                if (ni, nj) != (prev_i, prev_j) and \
                   0 <= ni < height and 0 <= nj < width and \
                   grid[ni][nj] != '#':
                    prev_i, prev_j = curr_i, curr_j
                    curr_i, curr_j = ni, nj
                    dist += 1
                    curr_dir = new_dir
                    found_next = True
                    break

            if not found_next:  # Dead end
                return (curr_i, curr_j), dist, curr_dir

    # Build graph
    graph = defaultdict(dict)
    nodes = [(i, j) for i in range(height) for j in range(width)
             if grid[i][j] in 'SE' or (grid[i][j] == '.' and is_intersection(i, j))]

    for node in nodes:
        i, j = node
        # Check all four directions
        for di, dj, direction in [(0, 1, 'E'), (1, 0, 'S'), (0, -1, 'W'), (-1, 0, 'N')]:
            ni, nj = i + di, j + dj
            if 0 <= ni < height and 0 <= nj < width and grid[ni][nj] != '#':
                next_node, dist, end_dir = find_next_node(ni, nj, i, j, direction)
                graph[node][next_node] = (dist + 1, end_dir)  # Just store the final direction

    return {
        'start': start,
        'end': end,
        'nodes': nodes,
        'graph': dict(graph)  # Convert defaultdict to regular dict
    }

def find_min_turn_path(maze_graph):
    """
    Find path from S to E minimizing turns (1000 points) and forward motion (1 point per step)
    Uses Dijkstra's algorithm with (node, facing_direction) as states
    """
    start = maze_graph['start']
    end = maze_graph['end']
    graph = maze_graph['graph']

    # Priority queue implemented as list for simplicity
    # Format: (total_cost, node, facing)
    queue = [(0, start, 'E')]  # Start facing East

    # Keep track of best costs and parents for path reconstruction
    # State is (node, facing)
    best_costs = {(start, 'E'): 0}
    parents = {}

    def turn_cost(from_dir: str, to_dir: str) -> int:
        """Calculate turning cost between directions"""
        if from_dir == to_dir:
            return 0
        # Opposite direction requires two turns
        opposites = {'N': 'S', 'S': 'N', 'E': 'W', 'W': 'E'}
        if opposites[from_dir] == to_dir:
            return 2000
        return 1000  # 90 degree turn

    while queue:
        # Get lowest cost state
        cost, node, facing = min(queue)
        queue.remove((cost, node, facing))

        # If we reached end, reconstruct path
        if node == end:
            path = []
            current = (node, facing)
            while current in parents:
                path.append(current)
                current = parents[current]
            path.append((start, 'E'))  # Add starting position
            path.reverse()
            return cost, path

        # Try all possible next moves
        for next_node, (dist, next_dir) in graph[node].items():
            # Calculate new cost including turn and distance
            new_cost = cost + turn_cost(facing, next_dir) + dist

            # If this is a better way to reach (next_node, next_dir)
            if (next_node, next_dir) not in best_costs or new_cost < best_costs[(next_node, next_dir)]:
                best_costs[(next_node, next_dir)] = new_cost
                parents[(next_node, next_dir)] = (node, facing)
                queue.append((new_cost, next_node, next_dir))

    return None  # No path found

result = solve_maze(input)
cost,_ = find_min_turn_path(result)
cost

134588